In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from keras import backend as K 
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Sequential
from keras.models import Model 
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Activation, Dropout, Flatten, Dense
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from keras.applications import InceptionResNetV2 
! pip install split-folders

import split_folders

Using TensorFlow backend.


In [0]:
DATA_PATH = 'gdrive/My Drive/NN-ProjectC/Project_C1/Training/'

# # Split the data into training and validation
# split_folders.ratio(DATA_PATH, output = "train_val", seed=1337, ratio = (.7,.3))

In [0]:
!ls "/content/drive/My Drive/NN-ProjectC/Project_C1/"

ls: cannot access '/content/drive/My Drive/NN-ProjectC/Project_C1/': No such file or directory


In [0]:
# Data Initialization
import os

! unzip "/content/drive/My Drive/ProjectC1/train_val_test_2.zip"

train_data_dir = "/content/train_val_test/content/train_val_test/train"
val_data_dir = "/content/train_val_test/content/train_val_test/val"
test_data_dir = "/content/train_val_test/content/train_val_test/test"

num_train = 0
num_val = 0
num_test = 0

for i in os.listdir(train_data_dir):
  num_train += len(os.listdir(os.path.join(train_data_dir, i)))
  
for i in os.listdir(val_data_dir):
  num_val += len(os.listdir(os.path.join(val_data_dir, i)))

for i in os.listdir(test_data_dir):
  num_test += len(os.listdir(os.path.join(test_data_dir, i)))

img_width, img_height = 224, 224
# Check if the images are RGB and change the channels likewise
if K.image_data_format() == 'channels_first':
  input_shape= (3, img_width, img_height)
else:
  input_shape = (img_width, img_height, 3)


In [0]:

# CNN model
model = Sequential()
model.add(Conv2D(32,(5,5), input_shape = input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

#model.add(Conv2D(32,(5,5)))
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(64,(5,5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

model.compile(loss = "categorical_crossentropy", optimizer = 'SGD', metrics = ['accuracy'])

In [0]:
batch_size = 64
epochs = 40
weight={0:1,1:3,2:4,3:4,4:6}
train_datagen = ImageDataGenerator(rescale = 1./255)
train_generator = train_datagen.flow_from_directory(train_data_dir, target_size =(img_width, img_height), batch_size = batch_size, class_mode = 'categorical')
validation_generator = train_datagen.flow_from_directory( val_data_dir, target_size =(img_width, img_height), batch_size = batch_size, class_mode ='categorical') 

model.fit_generator(train_generator, steps_per_epoch = num_train // batch_size, epochs = epochs, validation_data = validation_generator, validation_steps = num_val// batch_size)

model.save_weights("v1.h5")

Found 1481 images belonging to 5 classes.
Found 487 images belonging to 5 classes.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/40





23/23 [==============================] - 16s 714ms/step - loss: 1.6193 - acc: 0.2021 - val_loss: 1.5908 - val_acc: 0.2054
Epoch 2/40
23/23 [==============================] - 15s 636ms/step - loss: 1.5883 - acc: 0.2552 - val_loss: 1.6007 - val_acc: 0.1229
Epoch 3/40
23/23 [==============================] - 14s 627ms/step - loss: 1.5835 - acc: 0.2767 - val_loss: 1.5804 - val_acc: 0.1584
Epoch 4/40
23/23 [==============================] - 14s 617ms/step - loss: 1.5407 - acc: 0.2935 - val_loss: 1.5550 - val_acc: 0.3333
Epoch 5/40
23/23 [==============================] - 13s 582ms/step - loss: 1.5486 - acc: 0.3230 - val_loss: 1.5569 - val_acc: 0.3617
Epoch 6/40
23/23 [==============================] - 13s 577ms/step - loss: 1.4768 - acc: 0.4062 - val_loss: 1.4574 - val_acc: 0.2695
Epoch 7/40
23/23 

In [0]:
import numpy
import sklearn.metrics as metrics
img_width, img_height = 224, 224
batch_size = 64
epochs = 50

test_set_dir = "/content/train_val_test/content/train_val_test/test"

num_test = 0
for i in os.listdir(test_set_dir):
  num_test += len(os.listdir(os.path.join(test_set_dir, i)))

print ("Number of images in test set: ", num_test)

#model.load_weights("/content/v1.h5")
  
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_set_dir, target_size =(img_width, img_height), 
                                                  batch_size = batch_size, class_mode ='categorical') 
scoreSeg = model.evaluate_generator(test_generator, num_test // batch_size)
print(scoreSeg[1])

test_steps_per_epoch = numpy.math.ceil(test_generator.samples / test_generator.batch_size)

predictions = model.predict_generator(test_generator, steps = test_steps_per_epoch)
# Get most likely class
predicted_classes = numpy.argmax(predictions, axis=1)
true_classes = test_generator.classes
print (len(predicted_classes))
print (len(true_classes))
class_labels = list(test_generator.class_indices.keys()) 
report = metrics.confusion_matrix(true_classes, predicted_classes, labels=[0,1,2,3,4])

Number of images in test set:  264
Found 264 images belonging to 5 classes.
0.91796875
264
264


In [0]:
report

array([[10,  8,  9, 17, 13],
       [14,  9,  9, 13,  8],
       [ 6, 15, 16, 12,  8],
       [11, 13, 19, 10,  6],
       [ 4, 12, 10, 10,  2]])